# Generating Poincare Plots of Relative RR intervals (rr) and Harmonic Mean of RR intervals (hmrr)

This is the code implementation for the project "Detecting Cardiac Arrhythmia Using Poincaré plot of Heart Rate Variability (HRV) and Machine Learning".

In [1]:
# =================================================================
# Required packages
# =================================================================

import pandas as pd
import numpy as np
import wfdb
import os
import matplotlib.pyplot as plt


In [2]:
# =================================================================
# Loading real data from PhysioNet
# We first load the signal and rpeaks
# For one of the normal signal data set, we don't have the original signal, instead we have the RR-intervals
# =================================================================
   
def load_signal_func(record_name, file_format):
    tmp_ann_res = wfdb.rdann(path + '/' + record_name, file_format).__dict__
    fs=tmp_ann_res['fs']
    annotated=tmp_ann_res['sample']
    annotated=list(annotated)
    tmp_data_res = wfdb.rdsamp(path + '/' + record_name)
    ecg_signal=tmp_data_res[0][:, 1]    
    return fs, annotated, ecg_signal



def load_rr_int_func(record_name, file_format):
    tmp_ann_res = wfdb.rdann(path + '/' + record_name, file_format).__dict__
    fs=tmp_ann_res['fs']
    annotated=tmp_ann_res['sample']
    annotated=list(annotated)     
    return fs, annotated


In [3]:
#%%
# =================================================================
# Loading the data
# Calculate the rr and hmrr
# Save the Poincare plots of rr and hmrr
# =================================================================
    


file_list=['mit-bih-atrial-fibrillation-database-1.0.0', 'long-term-af-database-1.0.0', 'mit-bih-normal-sinus-rhythm-database-1.0.0', 'normal-sinus-rhythm-rr-interval-database-1.0.0']

for file in file_list:
    path = 'C:/Users/'+file+'/'
    print(path)

    # Find the name of all the records in the path       
    with open(os.path.join(path, 'RECORDS'), 'r') as fin:
        all_record_name = fin.read().strip().split('\n')
    
    
    # For saving the plots, need the signal_type
    if (file=='mit-bih-atrial-fibrillation-database-1.0.0') or (file=='long-term-af-database-1.0.0'):
        signal_type = 'af'
    else:
        signal_type = 'nsr'
    
    
    # The first two patients from 'mit-bih-atrial-fibrillation-database-1.0.0' are removed
    if file=='mit-bih-atrial-fibrillation-database-1.0.0':
        starting_patient=2
    else:
        starting_patient=0
       
        
    # The file formats are different   
    if file=='mit-bih-normal-sinus-rhythm-database-1.0.0':
        file_format='atr'
    elif file=='normal-sinus-rhythm-rr-interval-database-1.0.0':
        file_format='ecg'
    else:
        file_format='qrs'
    
    
    for i in range(starting_patient, len(all_record_name)):
        record_name = all_record_name[i]
        
        # Loading the data
        # Calculating RR intervals
        # for the dataset 'normal-sinus-rhythm-rr-interval-database-1.0.0', the signal is not available
        if file=='normal-sinus-rhythm-rr-interval-database-1.0.0':
            sampling_rate, rpeaks = load_rr_int_func(record_name, file_format)
            t = np.arange(0.0, np.max(rpeaks)+1) / sampling_rate
            rr_int = t[rpeaks[1:]] - t[rpeaks[:-1]]
        
        else:  
            sampling_rate, rpeaks, ecg_signal = load_signal_func(record_name, file_format)
            t = np.arange(0.0, ecg_signal.shape[0]) / sampling_rate
            rr_int = t[rpeaks[1:]] - t[rpeaks[:-1]]
        
        
        # There are some missing R peaks in between,
        # To resolve the issue, we applied a filter to remove the segments that R peaks are more than two seconds apart
        rr_int=rr_int[rr_int<2]
        
        # =================================================================
        # plotting the poincare plots
        # =================================================================
    
        #relative rr_int diff pairs
        rr_int_relative_diff=2*(rr_int[1:] - rr_int[:-1])/(rr_int[1:] + rr_int[:-1])
        fig, ax = plt.subplots(figsize=(6.5,6))
        plt.scatter(rr_int_relative_diff[:-1],rr_int_relative_diff[1:], color='blue')
        plt.xticks([])
        plt.yticks([])
        plt.xlim([-2,2])
        plt.ylim([-2,2])
        plt.savefig('C:/Users/rr_plots/%s_%s_relative_rr_int.jpeg'%(signal_type,record_name))
    
    
        #harmonicrr_int diff pairs
        rr_int_harmonic_diff=2*(rr_int[1:] * rr_int[:-1])/(rr_int[1:] + rr_int[:-1])
        fig, ax = plt.subplots(figsize=(6.5,6))
        plt.scatter(rr_int_harmonic_diff[:-1],rr_int_harmonic_diff[1:], color='blue')
        plt.xticks([])
        plt.yticks([])
        plt.xlim([0,2])
        plt.ylim([0,2])
        plt.savefig('C:/Users/hmrr_plots/%s_%s_harmonic_mean_of_rr_int.jpeg'%(signal_type,record_name))
